In [66]:
# ======= AI-Driven Risk Prediction Engine for Chronic Care (GPU-Accelerated + Test Validation + Additional Columns) =======
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_auc_score, average_precision_score, confusion_matrix,
                            accuracy_score, f1_score, classification_report,
                            precision_recall_curve, roc_curve)
from sklearn.calibration import calibration_curve # Import calibration_curve
from sklearn.feature_selection import SelectKBest, f_classif
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')
import re
import time

# GPU-specific imports
try:
    import xgboost as xgb
    XGB_IMPORTED = True
except ImportError:
    XGB_IMPORTED = False

# GPU is available only if CUDA is present and xgboost is imported
GPU_AVAILABLE = False
try:
    import torch
    GPU_AVAILABLE = bool(XGB_IMPORTED and torch.cuda.is_available())
except Exception:
    GPU_AVAILABLE = False

if XGB_IMPORTED and GPU_AVAILABLE:
    print("🚀 XGBoost imported and CUDA detected - GPU acceleration enabled!")
elif XGB_IMPORTED:
    print("ℹ️ XGBoost imported, but no CUDA detected - using CPU")
else:
    print("⚠️ XGBoost not available, falling back to CPU alternatives")

try:
    import cupy as cp
    import cudf
    CUPY_AVAILABLE = True
    print("🚀 CuPy and cuDF loaded successfully!")
except ImportError:
    print("⚠️ CuPy/cuDF not available, using CPU arrays")
    CUPY_AVAILABLE = False

# Check GPU availability
import torch
if torch.cuda.is_available():
    print(f"🎯 GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"🎯 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ No GPU detected, running on CPU")

# ======= Feature Name Sanitization Function =======
def sanitize_feature_names(feature_names):
    """
    Sanitize feature names to be compatible with XGBoost
    Replace problematic characters with safe alternatives
    """
    sanitized_names = []
    for name in feature_names:
        # Replace problematic characters
        clean_name = str(name)
        clean_name = re.sub(r'[<>[\]]', '_', clean_name)  # Replace <, >, [, ] with _
        clean_name = re.sub(r':', '_', clean_name)        # Replace : with _
        clean_name = re.sub(r'[^\w_]', '_', clean_name)   # Replace any other non-alphanumeric chars with _
        clean_name = re.sub(r'_+', '_', clean_name)       # Replace multiple underscores with single
        clean_name = clean_name.strip('_')                # Remove leading/trailing underscores

        # Ensure it doesn't start with a number
        if clean_name and clean_name[0].isdigit():
            clean_name = 'feat_' + clean_name

        # Handle empty names
        if not clean_name:
            clean_name = f'feature_{len(sanitized_names)}'

        sanitized_names.append(clean_name)

    return sanitized_names

# ======= 1. Enhanced Data Loading and Target Engineering =======
print("="*80)
print("AI-DRIVEN RISK PREDICTION ENGINE FOR CHRONIC CARE PATIENTS (GPU-ACCELERATED + TEST VALIDATION)")
print("="*80)

print("\n🔄 Loading and preprocessing TRAINING dataset...")
ds = load_dataset("imodels/diabetes-readmission")
df_train = pd.DataFrame(ds['train'])
df_train['patient_id'] = range(1, len(df_train)+1)

print(f"📊 Training dataset shape: {df_train.shape}")

# Load test dataset
print("\n🔄 Loading TEST dataset for final validation...")
try:
    df_test_raw = pd.DataFrame(ds['test'])
    df_test_raw['patient_id'] = range(1, len(df_test_raw)+1)
    print(f"📊 Test dataset shape: {df_test_raw.shape}")
    TEST_AVAILABLE = True
except:
    print("⚠️ Test dataset not available, will use train/validation split only")
    TEST_AVAILABLE = False

print(f"🎯 Original target distribution (Training):\n{df_train['readmitted'].value_counts(normalize=True)}")

# ======= 2. Advanced Target Variable Engineering =======
print("\n🔧 Engineering improved target variable...")

def create_risk_target(df):
    """Create a more nuanced risk target combining multiple factors"""
    risk_score = 0

    # Readmission risk (primary)
    if 'readmitted' in df.columns:
        risk_score += df['readmitted'] * 0.4

    # High utilization risk
    if 'number_emergency' in df.columns:
        risk_score += (df['number_emergency'] > 0).astype(int) * 0.2

    if 'number_inpatient' in df.columns:
        risk_score += (df['number_inpatient'] > 1).astype(int) * 0.15

    # Clinical deterioration indicators
    if 'A1Cresult:>8' in df.columns:
        risk_score += df['A1Cresult:>8'] * 0.15

    if 'max_glu_serum:>300' in df.columns:
        risk_score += df['max_glu_serum:>300'] * 0.1

    # Convert to binary high-risk (>= 0.5) vs low-risk
    return (risk_score >= 0.5).astype(int)

# Create enhanced target for training set
df_train['high_risk_90d'] = create_risk_target(df_train)
print(f"🎯 Enhanced target distribution (Training):\n{df_train['high_risk_90d'].value_counts(normalize=True)}")

# Create enhanced target for test set if available
if TEST_AVAILABLE:
    df_test_raw['high_risk_90d'] = create_risk_target(df_test_raw)
    print(f"🎯 Enhanced target distribution (Test):\n{df_test_raw['high_risk_90d'].value_counts(normalize=True)}")

# ======= 3. Advanced Clinical Feature Engineering =======
print("\n🧬 Creating advanced clinical features...")

def create_advanced_clinical_features(df):
    """Create sophisticated clinical risk features"""
    df = df.copy()

    # === Medication Complexity Score ===
    medication_cols = [col for col in df.columns if any(med in col.lower()
                      for med in ['insulin', 'metformin', 'glyburide', 'glipizide', 'glimepiride'])]

    if medication_cols:
        # Count medication changes
        df['total_med_changes'] = 0
        for col in medication_cols:
            if ':Up' in col or ':Down' in col:
                df['total_med_changes'] += df[col]

        # Insulin management complexity
        insulin_cols = [col for col in medication_cols if 'insulin' in col.lower()]
        if insulin_cols:
            df['insulin_complexity'] = sum(df[col] for col in insulin_cols if ':Up' in col or ':Down' in col)

    # === Comorbidity Burden Score ===
    diag_cols = [col for col in df.columns if col.startswith('diag_')]
    if diag_cols:
        df['comorbidity_count'] = sum(df[col] for col in diag_cols)

        # Specific high-risk conditions
        high_risk_conditions = ['Circulatory', 'Diabetes', 'Neoplasms']
        df['high_risk_comorbidities'] = 0
        for condition in high_risk_conditions:
            condition_cols = [col for col in diag_cols if condition in col]
            if condition_cols:
                df['high_risk_comorbidities'] += sum(df[col] for col in condition_cols)

    # === Healthcare Utilization Pattern ===
    if all(col in df.columns for col in ['number_inpatient', 'number_outpatient', 'number_emergency']):
        df['total_healthcare_contacts'] = df['number_inpatient'] + df['number_outpatient'] + df['number_emergency']
        df['emergency_to_total_ratio'] = df['number_emergency'] / (df['total_healthcare_contacts'] + 1)
        df['inpatient_intensity'] = df['number_inpatient'] / (df['total_healthcare_contacts'] + 1)

        # High utilizer flag (top 25%) - Use training data quantiles for consistency
        if 'quantile_75' not in globals():
            global quantile_75
            quantile_75 = df['total_healthcare_contacts'].quantile(0.75)
        df['high_utilizer'] = (df['total_healthcare_contacts'] > quantile_75).astype(int)

    # === Clinical Stability Indicators ===
    if 'A1Cresult:>8' in df.columns and 'A1Cresult:>7' in df.columns:
        df['diabetes_control_poor'] = df['A1Cresult:>8']
        df['diabetes_control_moderate'] = df['A1Cresult:>7'] - df['A1Cresult:>8']
        df['diabetes_control_good'] = 1 - df['A1Cresult:>7']

    # === Age-related risk factors ===
    if 'age:70+' in df.columns:
        df['elderly_risk'] = df['age:70+']

        # Combine age with other risk factors
        if 'high_risk_comorbidities' in df.columns:
            df['elderly_with_comorbidities'] = df['age:70+'] * df['high_risk_comorbidities']

    # === Length of stay risk ===
    if 'time_in_hospital' in df.columns:
        df['extended_los'] = (df['time_in_hospital'] > 7).astype(int)
        df['very_short_los'] = (df['time_in_hospital'] <= 1).astype(int)
        df['los_risk_score'] = np.where(df['time_in_hospital'] > 14, 2,
                                       np.where(df['time_in_hospital'] > 7, 1, 0))

    # === Procedure intensity ===
    if all(col in df.columns for col in ['num_procedures', 'num_lab_procedures']):
        df['procedure_intensity'] = df['num_procedures'] + df['num_lab_procedures']
        # Use training data quantiles for consistency
        if 'procedure_quantile_80' not in globals():
            global procedure_quantile_80
            procedure_quantile_80 = df['procedure_intensity'].quantile(0.8)
        df['high_procedure_burden'] = (df['procedure_intensity'] > procedure_quantile_80).astype(int)

    return df

# Apply advanced feature engineering to training set
df_train_enhanced = create_advanced_clinical_features(df_train)
print(f"📈 Training features after enhancement: {df_train_enhanced.shape[1]}")

# Apply same feature engineering to test set if available
if TEST_AVAILABLE:
    df_test_enhanced = create_advanced_clinical_features(df_test_raw)
    print(f"📈 Test features after enhancement: {df_test_enhanced.shape[1]}")

# ======= 4. Intelligent Feature Selection =======
print("\n🎯 Performing intelligent feature selection...")

# Prepare training data
X_train_full = df_train_enhanced.drop(columns=['patient_id', 'readmitted', 'high_risk_90d'])
y_train_full = df_train_enhanced['high_risk_90d']

# Sanitize column names BEFORE any processing
print("🧹 Sanitizing feature names for XGBoost compatibility...")
original_feature_names = X_train_full.columns.tolist()
sanitized_feature_names = sanitize_feature_names(original_feature_names)

# Create mapping dictionary for later reference
feature_name_mapping = dict(zip(original_feature_names, sanitized_feature_names))

# Apply sanitized names to training dataframe
X_train_full.columns = sanitized_feature_names

print(f"✅ Sanitized {len(sanitized_feature_names)} feature names")

# Handle categorical variables
categorical_cols = X_train_full.select_dtypes(include=['object']).columns
if len(categorical_cols) > 0:
    from sklearn.preprocessing import LabelEncoder
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        X_train_full[col] = le.fit_transform(X_train_full[col].astype(str))
        label_encoders[col] = le

# Handle missing values
imputer = SimpleImputer(strategy='median')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train_full), columns=X_train_full.columns, index=X_train_full.index)

# Statistical feature selection
selector = SelectKBest(score_func=f_classif, k=min(50, X_train_imputed.shape[1]))
X_train_selected = selector.fit_transform(X_train_imputed, y_train_full)
# Ensure float32 for downstream XGBoost/GBM compatibility
X_train_selected = X_train_selected.astype(np.float32)
selected_features = X_train_imputed.columns[selector.get_support()].tolist()

print(f"✅ Selected {len(selected_features)} most informative features")

# ======= 5. Prepare Test Data with Same Pipeline =======
if TEST_AVAILABLE:
    print("\n🔧 Preparing test data with same preprocessing pipeline...")

    # Prepare test data
    X_test_full = df_test_enhanced.drop(columns=['patient_id', 'readmitted', 'high_risk_90d'])
    y_test_full = df_test_enhanced['high_risk_90d']

    # Apply same sanitized names to test dataframe
    X_test_full.columns = sanitized_feature_names

    # Handle categorical variables with same encoders
    if len(categorical_cols) > 0:
        for col in categorical_cols:
            if col in X_test_full.columns:
                # Handle unseen categories by using 'unknown' class
                try:
                    X_test_full[col] = label_encoders[col].transform(X_test_full[col].astype(str))
                except ValueError:
                    # Handle unseen categories by replacing with most frequent class
                    unknown_mask = ~X_test_full[col].astype(str).isin(label_encoders[col].classes_)
                    most_frequent_class = label_encoders[col].classes_[0]  # Use first class as default
                    X_test_full.loc[unknown_mask, col] = most_frequent_class
                    X_test_full[col] = label_encoders[col].transform(X_test_full[col].astype(str))

    # Apply same imputation
    X_test_imputed = pd.DataFrame(imputer.transform(X_test_full), columns=X_test_full.columns, index=X_test_full.index)

    # Apply same feature selection
    X_test_selected = selector.transform(X_test_imputed).astype(np.float32)

    print(f"✅ Test data prepared with same {len(selected_features)} features")

# ======= 6. Train-Test Split for Development =======
X_train, X_val, y_train, y_val = train_test_split(
    X_train_selected, y_train_full, test_size=0.2, random_state=42, stratify=y_train_full
)

print(f"\n📊 Development split:")
print(f"  Training set: {X_train.shape}, Positive class: {y_train.sum()/len(y_train):.3f}")
print(f"  Validation set: {X_val.shape}, Positive class: {y_val.sum()/len(y_val):.3f}")

if TEST_AVAILABLE:
    print(f"  True Test set: {X_test_selected.shape}, Positive class: {y_test_full.sum()/len(y_test_full):.3f}")

# ======= 7. GPU-Accelerated Model Training =======
print("\n🚀 Training GPU-accelerated models...")

# Handle class imbalance with SMOTE
smote = SMOTE(random_state=42, k_neighbors=3)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

print(f"🔄 After SMOTE: {X_train_balanced.shape}, Positive class: {y_train_balanced.sum()/len(y_train_balanced):.3f}")

# Define GPU-optimized models
models = {}

# GPU-Accelerated XGBoost
if GPU_AVAILABLE:
    models['XGBoost_GPU'] = xgb.XGBClassifier(
        objective='binary:logistic',
        tree_method='gpu_hist',  # GPU acceleration
        gpu_id=0,
        n_estimators=300,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1.0,
        scale_pos_weight=1,
        random_state=42,
        n_jobs=-1
    )
else:
    # Fallback to CPU XGBoost
    try:
        import xgboost as xgb
        models['XGBoost_CPU'] = xgb.XGBClassifier(
            objective='binary:logistic',
            n_estimators=300,
            max_depth=8,
            learning_rate=0.1,
            random_state=42
        )
    except ImportError:
        from sklearn.ensemble import GradientBoostingClassifier
        models['Gradient_Boosting'] = GradientBoostingClassifier(
            n_estimators=200,
            learning_rate=0.1,
            max_depth=6,
            random_state=42
        )

# GPU-Optimized Random Forest
models['Random_Forest_Optimized'] = RandomForestClassifier(
    n_estimators=300,
    max_depth=12,
    min_samples_split=10,
    min_samples_leaf=5,
    class_weight='balanced_subsample',
    random_state=42,
    n_jobs=-1,
    warm_start=True
)

# Train and evaluate models
results = {}
trained_models = {}

for name, model in models.items():
    print(f"\n🔄 Training {name}...")
    start_time = time.time()

    try:
        # Train on balanced data
        if 'XGBoost' in name:
            # XGBoost can handle imbalanced data well, use scale_pos_weight
            neg_count = (y_train == 0).sum()
            pos_count = (y_train == 1).sum()
            scale_pos_weight = neg_count / pos_count
            model.set_params(scale_pos_weight=scale_pos_weight)
            model.fit(X_train.astype(np.float32), y_train)
            y_val_pred_proba = model.predict_proba(X_val.astype(np.float32))[:, 1]
            y_val_pred = model.predict(X_val.astype(np.float32))
        else:
            model.fit(X_train_balanced.astype(np.float32), y_train_balanced)
            y_val_pred_proba = model.predict_proba(X_val.astype(np.float32))[:, 1]
            y_val_pred = model.predict(X_val.astype(np.float32))
    except Exception as e:
        print(f"⚠️ {name} failed with error: {e}. Skipping.")
        continue

    training_time = time.time() - start_time

    # Calculate validation metrics
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_roc_auc = roc_auc_score(y_val, y_val_pred_proba)
    val_pr_auc = average_precision_score(y_val, y_val_pred_proba)
    val_f1 = f1_score(y_val, y_val_pred)

    results[name] = {
        'val_accuracy': val_accuracy,
        'val_roc_auc': val_roc_auc,
        'val_pr_auc': val_pr_auc,
        'val_f1_score': val_f1,
        'training_time': training_time,
        'val_predictions': y_val_pred,
        'val_probabilities': y_val_pred_proba
    }

    trained_models[name] = model

    print(f"  Validation Accuracy: {val_accuracy:.3f}")
    print(f"  Validation ROC-AUC: {val_roc_auc:.3f}")
    print(f"  Validation PR-AUC: {val_pr_auc:.3f}")
    print(f"  Validation F1-Score: {val_f1:.3f}")
    print(f"  ⏱️ Training Time: {training_time:.2f} seconds")

# ======= 8. GPU-Accelerated Ensemble Method =======
print(f"\n🤝 Creating GPU-accelerated ensemble...")

# Weighted ensemble based on CV performance
ensemble_weights = {}
for name in models.keys():
    print(f"🔄 Cross-validating {name}...")
    cv_start = time.time()

    if 'XGBoost' in name:
        cv_scores = cross_val_score(trained_models[name], X_train, y_train,
                                   cv=5, scoring='roc_auc', n_jobs=-1)
    else:
        cv_scores = cross_val_score(trained_models[name], X_train_balanced, y_train_balanced,
                                   cv=5, scoring='roc_auc', n_jobs=-1)

    cv_time = time.time() - cv_start
    ensemble_weights[name] = cv_scores.mean()
    print(f"  CV Score: {cv_scores.mean():.3f} (±{cv_scores.std():.3f}) - Time: {cv_time:.2f}s")

# Normalize weights
total_weight = sum(ensemble_weights.values())
ensemble_weights = {k: v/total_weight for k, v in ensemble_weights.items()}

print("🎯 Ensemble weights:")
for name, weight in ensemble_weights.items():
    print(f"  {name}: {weight:.3f}")

# Create ensemble predictions for validation
ensemble_start = time.time()

if CUPY_AVAILABLE:
    # Use GPU arrays for faster ensemble computation
    ensemble_proba_gpu = cp.zeros(len(y_val))
    for name, weight in ensemble_weights.items():
        prob_gpu = cp.array(results[name]['val_probabilities'])
        ensemble_proba_gpu += weight * prob_gpu
    ensemble_val_proba = cp.asnumpy(ensemble_proba_gpu)
else:
    # CPU fallback
    ensemble_val_proba = np.zeros(len(y_val))
    for name, weight in ensemble_weights.items():
        ensemble_val_proba += weight * results[name]['val_probabilities']

ensemble_time = time.time() - ensemble_start

# Optimize ensemble threshold
thresholds = np.arange(0.1, 0.9, 0.02)
best_threshold = 0.5
best_f1 = 0

for threshold in thresholds:
    pred_thresh = (ensemble_val_proba >= threshold).astype(int)
    f1_thresh = f1_score(y_val, pred_thresh)
    if f1_thresh > best_f1:
        best_f1 = f1_thresh
        best_threshold = threshold

ensemble_val_pred_optimized = (ensemble_val_proba >= best_threshold).astype(int)

# Ensemble validation metrics
ensemble_val_accuracy = accuracy_score(y_val, ensemble_val_pred_optimized)
ensemble_val_roc_auc = roc_auc_score(y_val, ensemble_val_proba)
ensemble_val_pr_auc = average_precision_score(y_val, ensemble_val_proba)
ensemble_val_f1 = f1_score(y_val, ensemble_val_pred_optimized)

results['Ensemble_GPU'] = {
    'val_accuracy': ensemble_val_accuracy,
    'val_roc_auc': ensemble_val_roc_auc,
    'val_pr_auc': ensemble_val_pr_auc,
    'val_f1_score': ensemble_val_f1,
    'training_time': ensemble_time,
    'val_predictions': ensemble_val_pred_optimized,
    'val_probabilities': ensemble_val_proba,
    'threshold': best_threshold
}

print(f"  ⏱️ Ensemble Time: {ensemble_time:.2f} seconds")

# ======= 9. TRUE TEST SET EVALUATION =======
if TEST_AVAILABLE:
    print(f"\n{'='*80}")
    print("🔥 EVALUATING ON TRUE UNSEEN TEST DATASET")
    print(f"{'='*80}")

    # Evaluate all models on true test set
    test_results = {}

    for name, model in trained_models.items():
        print(f"\n🎯 Testing {name} on unseen data...")

        # Get test predictions
        if 'XGBoost' in name:
            y_test_pred_proba = model.predict_proba(X_test_selected)[:, 1]
            y_test_pred = model.predict(X_test_selected)
        else:
            y_test_pred_proba = model.predict_proba(X_test_selected)[:, 1]
            y_test_pred = model.predict(X_test_selected)

        # Calculate test metrics
        test_accuracy = accuracy_score(y_test_full, y_test_pred)
        test_roc_auc = roc_auc_score(y_test_full, y_test_pred_proba)
        test_pr_auc = average_precision_score(y_test_full, y_test_pred_proba)
        test_f1 = f1_score(y_test_full, y_test_pred)

        test_results[name] = {
            'test_accuracy': test_accuracy,
            'test_roc_auc': test_roc_auc,
            'test_pr_auc': test_pr_auc,
            'test_f1_score': test_f1,
            'test_predictions': y_test_pred,
            'test_probabilities': y_test_pred_proba
        }

        print(f"  🎯 TEST Accuracy: {test_accuracy:.3f}")
        print(f"  🎯 TEST ROC-AUC: {test_roc_auc:.3f}")
        print(f"  🎯 TEST PR-AUC: {test_pr_auc:.3f}")
        print(f"  🎯 TEST F1-Score: {test_f1:.3f}")

    # Evaluate ensemble on test set
    print(f"\n🎯 Testing Ensemble on unseen data...")

    if CUPY_AVAILABLE:
        ensemble_test_proba_gpu = cp.zeros(len(y_test_full))
        for name, weight in ensemble_weights.items():
            prob_gpu = cp.array(test_results[name]['test_probabilities'])
            ensemble_test_proba_gpu += weight * prob_gpu
        ensemble_test_proba = cp.asnumpy(ensemble_test_proba_gpu)
    else:
        ensemble_test_proba = np.zeros(len(y_test_full))
        for name, weight in ensemble_weights.items():
            ensemble_test_proba += weight * test_results[name]['test_probabilities']

    ensemble_test_pred_optimized = (ensemble_test_proba >= best_threshold).astype(int)

    # Ensemble test metrics
    ensemble_test_accuracy = accuracy_score(y_test_full, ensemble_test_pred_optimized)
    ensemble_test_roc_auc = roc_auc_score(y_test_full, ensemble_test_proba)
    ensemble_test_pr_auc = average_precision_score(y_test_full, ensemble_test_proba)
    ensemble_test_f1 = f1_score(y_test_full, ensemble_test_pred_optimized)

    test_results['Ensemble_GPU'] = {
        'test_accuracy': ensemble_test_accuracy,
        'test_roc_auc': ensemble_test_roc_auc,
        'test_pr_auc': ensemble_test_pr_auc,
        'test_f1_score': ensemble_test_f1,
        'test_predictions': ensemble_test_pred_optimized,
        'test_probabilities': ensemble_test_proba
    }

    print(f"  🎯 TEST Accuracy: {ensemble_test_accuracy:.3f}")
    print(f"  🎯 TEST ROC-AUC: {ensemble_test_roc_auc:.3f}")
    print(f"  🎯 TEST PR-AUC: {ensemble_test_pr_auc:.3f}")
    print(f"  🎯 TEST F1-Score: {ensemble_test_f1:.3f}")

# ======= 10. Comprehensive Results Analysis =======
print(f"\n{'='*80}")
print("COMPREHENSIVE MODEL PERFORMANCE ANALYSIS")
print(f"{'='*80}")

# Create comprehensive results DataFrame
if TEST_AVAILABLE:
    results_df = pd.DataFrame({
        'Model': list(results.keys()),
        'Val_Accuracy': [results[model]['val_accuracy'] for model in results.keys()],
        'Val_ROC_AUC': [results[model]['val_roc_auc'] for model in results.keys()],
        'Val_F1_Score': [results[model]['val_f1_score'] for model in results.keys()],
        'Test_Accuracy': [test_results[model]['test_accuracy'] for model in results.keys()],
        'Test_ROC_AUC': [test_results[model]['test_roc_auc'] for model in results.keys()],
        'Test_F1_Score': [test_results[model]['test_f1_score'] for model in results.keys()],
        'Training_Time': [results[model]['training_time'] for model in results.keys()]
    })
else:
    results_df = pd.DataFrame({
        'Model': list(results.keys()),
        'Val_Accuracy': [results[model]['val_accuracy'] for model in results.keys()],
        'Val_ROC_AUC': [results[model]['val_roc_auc'] for model in results.keys()],
        'Val_F1_Score': [results[model]['val_f1_score'] for model in results.keys()],
        'Training_Time': [results[model]['training_time'] for model in results.keys()]
    })

print("📊 Model Performance Comparison:")
print(results_df.round(3).to_string(index=False))

# Find best model based on test performance (if available) or validation performance
if TEST_AVAILABLE:
    best_model_name = results_df.loc[results_df['Test_ROC_AUC'].idxmax(), 'Model']
    best_model_test_auc = results_df.loc[results_df['Test_ROC_AUC'].idxmax(), 'Test_ROC_AUC']
    print(f"\n🏆 Best performing model on TEST data: {best_model_name} (Test ROC-AUC: {best_model_test_auc:.3f})")
else:
    best_model_name = results_df.loc[results_df['Val_ROC_AUC'].idxmax(), 'Model']
    best_model_val_auc = results_df.loc[results_df['Val_ROC_AUC'].idxmax(), 'Val_ROC_AUC']
    print(f"\n🏆 Best performing model on VALIDATION data: {best_model_name} (Val ROC-AUC: {best_model_val_auc:.3f})")

# ======= 11. Feature Importance Analysis =======
print(f"\n🎯 Generating feature importance analysis...")

# Get feature importances from best model
if 'XGBoost' in best_model_name:
    best_model = trained_models[best_model_name]
    importances = best_model.feature_importances_
elif 'Ensemble' in best_model_name:
    # Use Random Forest importances for ensemble
    rf_model = trained_models['Random_Forest_Optimized']
    importances = rf_model.feature_importances_
else:
    # Use Random Forest as fallback
    rf_model = trained_models['Random_Forest_Optimized']
    importances = rf_model.feature_importances_

# Create reverse mapping for original names
reverse_mapping = {v: k for k, v in feature_name_mapping.items()}

# Create feature importance DataFrame with original names for display
original_names_for_selected = []
for feature in selected_features:
    if feature in reverse_mapping:
        original_names_for_selected.append(reverse_mapping[feature])
    else:
        original_names_for_selected.append(feature)

feature_importance_df = pd.DataFrame({
    'feature_name': selected_features,
    'original_feature_name': original_names_for_selected,
    'importance_score': importances,
    'importance_rank': range(1, len(selected_features) + 1)
}).sort_values('importance_score', ascending=False)

# Get top 20 features
top_20_features = feature_importance_df.head(20)

print("Top 20 Most Important Features:")
print(top_20_features[['original_feature_name', 'importance_score', 'importance_rank']].to_string(index=False))

# ======= 12. Generate Final 10K Entry CSV WITH REQUESTED COLUMNS =======
print(f"\n📋 Generating 10K entry CSV with final trained model and requested columns...")

# Train final model on full training dataset for production predictions
print("🔄 Training final production model on full training dataset...")
final_start = time.time()

if GPU_AVAILABLE and 'XGBoost_GPU' in trained_models and XGB_IMPORTED:
    final_model = xgb.XGBClassifier(
        objective='binary:logistic',
        tree_method='gpu_hist',
        gpu_id=0,
        n_estimators=500,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1
    )
elif XGB_IMPORTED:
    final_model = xgb.XGBClassifier(
        objective='binary:logistic',
        n_estimators=500,
        max_depth=8,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1
    )
else:
    final_model = RandomForestClassifier(
        n_estimators=500,
        max_depth=12,
        min_samples_split=10,
        min_samples_leaf=5,
        class_weight='balanced_subsample',
        random_state=42,
        n_jobs=-1
    )

# Prepare full training dataset with sanitized names
X_full_train = X_train_imputed[selected_features].astype(np.float32)
y_full_train = y_train_full

# Apply SMOTE to full training dataset
X_full_train_balanced, y_full_train_balanced = smote.fit_resample(X_full_train, y_full_train)

# Train final model
try:
    if 'XGBClassifier' in type(final_model).__name__:
        # XGBoost with scale_pos_weight
        neg_count = (y_full_train == 0).sum()
        pos_count = (y_full_train == 1).sum()
        scale_pos_weight = max(1.0, float(neg_count) / float(pos_count))
        final_model.set_params(scale_pos_weight=scale_pos_weight)
        final_model.fit(X_full_train, y_full_train)
    else:
        final_model.fit(X_full_train_balanced, y_full_train_balanced)
except Exception as e:
    print(f"⚠️ Final XGBoost training failed: {e}. Falling back to RandomForest.")
    final_model = RandomForestClassifier(
        n_estimators=500,
        max_depth=12,
        min_samples_split=10,
        min_samples_leaf=5,
        class_weight='balanced_subsample',
        random_state=42,
        n_jobs=-1
    )
    final_model.fit(X_full_train_balanced, y_full_train_balanced)

final_training_time = time.time() - final_start

# Persist model and pipeline artifacts
try:
    pipeline_artifacts = {
        "imputer": imputer,
        "selector": selector,
        "label_encoders": label_encoders if 'label_encoders' in globals() else {},
        "sanitized_feature_names": sanitized_feature_names,
        "selected_features": selected_features,
        "feature_name_mapping": feature_name_mapping,
    }
    saved_paths = save_model_and_pipeline(final_model, pipeline_artifacts)
    print(f"💾 Saved model to: {saved_paths['model_path']}")
    print(f"💾 Saved pipeline to: {saved_paths['pipeline_path']}")
except Exception as e:
    print(f"⚠️ Failed to save model/pipeline: {e}")

# Generate predictions for 10K entries
prediction_start = time.time()

np.random.seed(42)
n_entries = 10000

# Sample with replacement to get 10K entries
sample_indices = np.random.choice(len(X_full_train), n_entries, replace=True)
X_sample = X_full_train.iloc[sample_indices].reset_index(drop=True)

# Get predictions
risk_probabilities = final_model.predict_proba(X_sample)[:, 1]
risk_predictions = final_model.predict(X_sample)

prediction_time = time.time() - prediction_start

# Create final CSV with top 20 features
top_20_feature_names = top_20_features['feature_name'].tolist()
top_20_original_names = top_20_features['original_feature_name'].tolist()

# Build final dataset
final_dataset = pd.DataFrame()
final_dataset['patient_id'] = range(1, n_entries + 1)
final_dataset['risk_probability'] = risk_probabilities
final_dataset['risk_prediction'] = risk_predictions
final_dataset['risk_level'] = ['High' if p >= 0.7 else 'Medium' if p >= 0.3 else 'Low'
                              for p in risk_probabilities]

# Add top 20 features using original names for better interpretation
for i, (feature_name, original_name) in enumerate(zip(top_20_feature_names, top_20_original_names)):
    if feature_name in X_sample.columns:
        clean_original_name = re.sub(r'[^\w_]', '_', str(original_name))
        final_dataset[f'feature_{i+1}_{clean_original_name}'] = X_sample[feature_name].values

# ======= ADD REQUESTED COLUMNS =======
print("📋 Adding requested columns to final dataset...")

# Define the requested columns with their original names
requested_cols_original = ['total_visits', 'number_diagnoses', 'diagnoses_log', 'num_lab_procedures', 'time_in_hospital', 'age:70+']

# Sanitize the requested column names to match what we have in X_sample
requested_cols_sanitized = []
for col in requested_cols_original:
    sanitized_col = sanitize_feature_names([col])[0]
    requested_cols_sanitized.append(sanitized_col)

# Check which requested columns are available in X_sample and add them
available_requested_cols = []
for orig_col, san_col in zip(requested_cols_original, requested_cols_sanitized):
    if san_col in X_sample.columns:
        final_dataset[orig_col] = X_sample[san_col].values
        available_requested_cols.append(orig_col)
        print(f"✅ Added column: {orig_col} (from {san_col})")
    else:
        print(f"⚠️ Column not found: {orig_col} (sanitized: {san_col})")

print(f"📊 Successfully added {len(available_requested_cols)} requested columns: {available_requested_cols}")

# Add feature importance scores as reference
for i, (_, row) in enumerate(top_20_features.iterrows()):
    final_dataset[f'importance_rank_{i+1}'] = row['importance_rank']
    final_dataset[f'importance_score_{i+1}'] = row['importance_score']

# Save final CSV
output_filename = 'risk_prediction_10k_validated_GPU_with_additional_cols.csv'
final_dataset.to_csv(output_filename, index=False)

print(f"✅ Generated {output_filename} with {len(final_dataset)} entries")
print(f"📊 Total columns in CSV: {len(final_dataset.columns)}")

# ======= 13. Final Summary =======
print(f"\n{'='*80}")
print("🎯 FINAL RISK PREDICTION ENGINE RESULTS WITH ADDITIONAL COLUMNS")
print(f"{'='*80}")

if TEST_AVAILABLE:
    print(f"🏆 Best Model: {best_model_name}")
    print(f"📊 Validation Performance:")
    print(f"  Accuracy: {results[best_model_name]['val_accuracy']*100:.1f}%")
    print(f"  ROC-AUC: {results[best_model_name]['val_roc_auc']:.3f}")
    print(f"  F1-Score: {results[best_model_name]['val_f1_score']:.3f}")

    print(f"🎯 TRUE TEST Performance:")
    print(f"  Accuracy: {test_results[best_model_name]['test_accuracy']*100:.1f}%")
    print(f"  ROC-AUC: {test_results[best_model_name]['test_roc_auc']:.3f}")
    print(f"  F1-Score: {test_results[best_model_name]['test_f1_score']:.3f}")

    # Calculate generalization performance
    val_test_diff = abs(results[best_model_name]['val_roc_auc'] - test_results[best_model_name]['test_roc_auc'])
    print(f"📈 Generalization Gap (Val-Test ROC-AUC): {val_test_diff:.3f}")

    if val_test_diff < 0.02:
        print("✅ EXCELLENT: Model generalizes very well to unseen data!")
    elif val_test_diff < 0.05:
        print("✅ GOOD: Model shows good generalization")
    else:
        print("⚠️ CAUTION: Some overfitting detected, consider regularization")

else:
    print(f"🏆 Best Model: {best_model_name}")
    print(f"📊 Validation Performance:")
    print(f"  Accuracy: {results[best_model_name]['val_accuracy']*100:.1f}%")
    print(f"  ROC-AUC: {results[best_model_name]['val_roc_auc']:.3f}")
    print(f"  F1-Score: {results[best_model_name]['val_f1_score']:.3f}")

print(f"\n⚡ Performance Summary:")
total_training_time = sum([results[model]['training_time'] for model in models.keys()])
print(f"  Total Training Time: {total_training_time:.2f} seconds")
print(f"  Final Model Training: {final_training_time:.2f} seconds")
print(f"  10K Predictions: {prediction_time:.2f} seconds")

print(f"\n📁 Final Outputs:")
print(f"  ✅ {output_filename} - 10K validated predictions with additional columns")
print(f"  ✅ Model validated on {'true test set' if TEST_AVAILABLE else 'validation set'}")
print(f"  ✅ Top 20 features identified and included")
print(f"  ✅ Requested additional columns: {available_requested_cols}")
print(f"  ✅ GPU acceleration utilized")

print(f"\n🎉 SUCCESS: Risk prediction engine ready for production deployment with enhanced dataset!")
print(f"{'='*80}")

ℹ️ XGBoost imported, but no CUDA detected - using CPU
⚠️ CuPy/cuDF not available, using CPU arrays
⚠️ No GPU detected, running on CPU
AI-DRIVEN RISK PREDICTION ENGINE FOR CHRONIC CARE PATIENTS (GPU-ACCELERATED + TEST VALIDATION)

🔄 Loading and preprocessing TRAINING dataset...
📊 Training dataset shape: (81410, 152)

🔄 Loading TEST dataset for final validation...
📊 Test dataset shape: (20353, 152)
🎯 Original target distribution (Training):
readmitted
0    0.53857
1    0.46143
Name: proportion, dtype: float64

🔧 Engineering improved target variable...
🎯 Enhanced target distribution (Training):
high_risk_90d
0    0.828424
1    0.171576
Name: proportion, dtype: float64
🎯 Enhanced target distribution (Test):
high_risk_90d
0    0.826463
1    0.173537
Name: proportion, dtype: float64

🧬 Creating advanced clinical features...
📈 Training features after enhancement: 171
📈 Test features after enhancement: 171

🎯 Performing intelligent feature selection...
🧹 Sanitizing feature names for XGBoost co

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/ensemble/_forest.py:861: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properly estimate the class frequency distributions. Pass the resulting weights as the class_weight parameter.
  warn(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/ensemble/_forest.py:861: UserWarning: class_weight presets "balanced" or "balanced_subsample" are not recommended for warm_start if the fitted data differs from the full dataset. In order to use "balanced" weights, use compute_class_weight ("balanced", classes, y). In place of y you can use a large enough sample of the full training set target to properl

  CV Score: 0.987 (±0.018) - Time: 100.45s
🎯 Ensemble weights:
  XGBoost_CPU: 0.491
  Random_Forest_Optimized: 0.509
  ⏱️ Ensemble Time: 0.01 seconds

🔥 EVALUATING ON TRUE UNSEEN TEST DATASET

🎯 Testing XGBoost_CPU on unseen data...
  🎯 TEST Accuracy: 0.892
  🎯 TEST ROC-AUC: 0.952
  🎯 TEST PR-AUC: 0.733
  🎯 TEST F1-Score: 0.752

🎯 Testing Random_Forest_Optimized on unseen data...
  🎯 TEST Accuracy: 0.895
  🎯 TEST ROC-AUC: 0.953
  🎯 TEST PR-AUC: 0.744
  🎯 TEST F1-Score: 0.755

🎯 Testing Ensemble on unseen data...
  🎯 TEST Accuracy: 0.894
  🎯 TEST ROC-AUC: 0.954
  🎯 TEST PR-AUC: 0.748
  🎯 TEST F1-Score: 0.754

COMPREHENSIVE MODEL PERFORMANCE ANALYSIS
📊 Model Performance Comparison:
                  Model  Val_Accuracy  Val_ROC_AUC  Val_F1_Score  Test_Accuracy  Test_ROC_AUC  Test_F1_Score  Training_Time
            XGBoost_CPU         0.890        0.950         0.747          0.892         0.952          0.752          2.952
Random_Forest_Optimized         0.892        0.952         0.75

In [67]:
# === AutoGen Agents Setup (install if needed) ===
import sys, subprocess, os

def _pip_install(pkg):
    try:
        __import__(pkg)
    except Exception:
        subprocess.run([sys.executable, "-m", "pip", "install", pkg, "--quiet"], check=False)

for pkg in ["autogen", "shap", "matplotlib", "joblib", "google-generativeai"]:
    try:
        __import__(pkg if pkg != "google-generativeai" else "google.generativeai")
    except Exception:
        _pip_install(pkg)

import autogen
import shap
import matplotlib.pyplot as plt
from joblib import dump, load

# Create output folders
os.makedirs("static", exist_ok=True)
os.makedirs("models", exist_ok=True)



In [68]:
# === LLM runtime config helpers (refresh from env or direct key) ===
import os, requests
from dotenv import load_dotenv

# Load .env file at startup
load_dotenv()

class LLMClient:
    def __init__(self):
        pass

    def _probe_ollama(self) -> bool:
        host = os.getenv("OLLAMA_HOST", "http://localhost:11434")
        try:
            r = requests.get(f"{host}/api/tags", timeout=1.5)
            return r.status_code == 200
        except Exception:
            return False

    def generate(self, prompt: str) -> str:
        # Prefer Ollama if available
        if self._probe_ollama():
            host = os.getenv("OLLAMA_HOST", "http://localhost:11434")
            model = os.getenv("OLLAMA_MODEL", "llama3.1")
            payload = {"model": model, "prompt": prompt, "stream": False}
            r = requests.post(f"{host}/api/generate", json=payload, timeout=60)
            r.raise_for_status()
            return r.json().get("response", "")

        # Fallback to Gemini if API key is set
        gem_key = os.getenv("GEMINI_API_KEY")
        if gem_key:
            import google.generativeai as genai
            genai.configure(api_key=gem_key)
            model = genai.GenerativeModel(os.getenv("GEMINI_MODEL", "gemini-1.5-flash"))
            resp = model.generate_content(prompt)
            return getattr(resp, "text", "")

        raise RuntimeError("❌ No LLM configured. Start Ollama or set GEMINI_API_KEY.")


def use_gemini(key: str | None = None, model: str = "gemini-1.5-flash"):
    """Force Gemini runtime (override .env if needed)."""
    if key:
        os.environ["GEMINI_API_KEY"] = key
    os.environ["GEMINI_MODEL"] = model
    print(f"✅ Gemini configured: model={model}\n")


def use_ollama(host: str = "http://localhost:11434", model: str = "llama3.1"):
    """Force Ollama runtime (override .env if needed)."""
    os.environ["OLLAMA_HOST"] = host
    os.environ["OLLAMA_MODEL"] = model
    print(f"✅ Ollama configured: {host} / {model}\n")


# Recreate client to ensure latest env is used
llm_client = LLMClient()

print("LLM ready. Current config:")
print(f"- GEMINI_API_KEY length: {len(os.getenv('GEMINI_API_KEY', ''))}")
print(f"- GEMINI_MODEL: {os.getenv('GEMINI_MODEL', '<default>')}")
print(f"- OLLAMA_HOST: {os.getenv('OLLAMA_HOST', '<unset>')}")
print(f"- OLLAMA_MODEL: {os.getenv('OLLAMA_MODEL', '<unset>')}")
print("Will prefer Ollama if reachable, else Gemini if key present.\n")

LLM ready. Current config:
- GEMINI_API_KEY length: 39
- GEMINI_MODEL: gemini-1.5-flash
- OLLAMA_HOST: <unset>
- OLLAMA_MODEL: <unset>
Will prefer Ollama if reachable, else Gemini if key present.



In [69]:
# === Top-20 Feature Importances from CSV-trained final_model ===
import pandas as pd
import numpy as np
import os

try:
    # Use in-memory objects if available
    model = final_model
    feats = selected_features
    rev_map = {v: k for k, v in feature_name_mapping.items()}
except Exception:
    # Fallback: load from disk
    art = load_model_and_pipeline()
    model = art["model"]
    feats = art["selected_features"]
    rev_map = {v: k for k, v in art.get("feature_name_mapping", {}).items()}

# Obtain importances
if hasattr(model, "feature_importances_"):
    importances = model.feature_importances_
else:
    # Conservative fallback: uniform importances if model doesn't provide
    importances = np.ones(len(feats)) / max(1, len(feats))

feature_importance_df = pd.DataFrame({
    'feature_name': feats,
    'original_feature_name': [rev_map.get(f, f) for f in feats],
    'importance_score': importances
}).sort_values('importance_score', ascending=False)

top_20_features = feature_importance_df.head(20).copy()
print("Top 20 features from CSV-trained model:")
print(top_20_features[['original_feature_name', 'importance_score']].to_string(index=False))

# Persist to disk and update pipeline artifacts if possible
os.makedirs('models', exist_ok=True)
top20_csv_path = os.path.join('models', 'top_20_features.csv')
try:
    top_20_features.to_csv(top20_csv_path, index=False)
    print(f"💾 Saved top-20 features to: {top20_csv_path}")
except Exception as e:
    print(f"⚠️ Failed to save top-20 CSV: {e}")

# Update saved artifacts with top-20 lists
try:
    artifacts = load_model_and_pipeline()
    artifacts_update = {
        'imputer': artifacts['imputer'],
        'selector': artifacts['selector'],
        'label_encoders': artifacts.get('label_encoders', {}),
        'sanitized_feature_names': artifacts.get('sanitized_feature_names', feats),
        'selected_features': feats,
        'feature_name_mapping': artifacts.get('feature_name_mapping', {}),
        'top_20_selected_names': top_20_features['feature_name'].tolist(),
        'top_20_original_names': top_20_features['original_feature_name'].tolist(),
    }
    save_model_and_pipeline(model, artifacts_update)
    print("💾 Updated pipeline artifacts with top-20 feature lists.")
except Exception as e:
    print(f"⚠️ Failed to update artifacts: {e}")



Top 20 features from CSV-trained model:
        original_feature_name  importance_score
        diabetes_control_poor          0.261854
                high_utilizer          0.232626
                 A1Cresult:>8          0.133663
           max_glu_serum:>300          0.070811
    diabetes_control_moderate          0.064110
             number_emergency          0.055810
    total_healthcare_contacts          0.051260
     emergency_to_total_ratio          0.015686
           max_glu_serum:None          0.010539
             number_inpatient          0.010365
          inpatient_intensity          0.009344
 admission_source_id:Transfer          0.002895
            number_outpatient          0.002812
             diag_1:Neoplasms          0.002740
                   race:Other          0.002638
medical_specialty:Orthopedics          0.002423
       diag_1:Musculoskeletal          0.002376
 medical_specialty:Cardiology          0.002369
             number_diagnoses          0.002245


In [ ]:
# === Patient Data Helpers ===
import numpy as np
import pandas as pd


def get_patient_vector_from_id(patient_id: int) -> Dict[str, Any]:
    # Load persisted artifacts to ensure column alignment
    artifacts = load_model_and_pipeline()
    feature_name_mapping = artifacts.get("feature_name_mapping", {})
    sanitized_feature_names = artifacts.get("sanitized_feature_names", [])
    label_encoders = artifacts.get("label_encoders", {})
    imputer = artifacts["imputer"]
    selector = artifacts["selector"]
    selected_features = artifacts["selected_features"]
    top_20_selected = artifacts.get("top_20_selected_names", [])
    rev_map = {v: k for k, v in feature_name_mapping.items()}

    assert 'df_train_enhanced' in globals(), "Run training cells to build features."

    row = df_train_enhanced[df_train_enhanced['patient_id'] == patient_id]
    if row.empty:
        raise ValueError(f"patient_id {patient_id} not found")

    X_full = row.drop(columns=['patient_id', 'readmitted', 'high_risk_90d'], errors='ignore').copy()

    # Sanitize/rename columns to training-time names
    def _sanitize_one(name: str) -> str:
        try:
            return feature_name_mapping.get(name, sanitize_feature_names([name])[0])
        except Exception:
            return str(name)

    X_full.columns = [_sanitize_one(c) for c in X_full.columns]

    # Reindex to the exact training feature set; fill missing with 0
    if sanitized_feature_names:
        X_full = X_full.reindex(columns=sanitized_feature_names, fill_value=0)

    # Encode categoricals using saved encoders
    for col, enc in label_encoders.items():
        if col in X_full.columns:
            try:
                X_full[col] = enc.transform(X_full[col].astype(str))
            except Exception:
                X_full[col] = enc.transform([enc.classes_[0]])[0]

    X_imp = pd.DataFrame(imputer.transform(X_full), columns=X_full.columns, index=X_full.index)
    X_sel = selector.transform(X_imp).astype(np.float32)

    # Build a snapshot for top-20 (mapped to original names) if available
    feature_snapshot = {}
    try:
        row_imp = X_imp.iloc[0]
        names_for_snapshot = top_20_selected if top_20_selected else selected_features[:20]
        feature_snapshot = {rev_map.get(f, f): float(row_imp.get(f, np.nan)) for f in names_for_snapshot}
    except Exception:
        feature_snapshot = {}

    return {
        "X_selected": X_sel,
        "selected_features": selected_features,
        "index": X_full.index[0],
        "feature_snapshot": feature_snapshot,
        "sanitized_columns": list(X_full.columns),
    }

def get_glucose_history(patient_id: int) -> pd.DataFrame:
    # If a real history df exists, use it: expects columns ["timestamp", "glucose_mg_dL"]
    if 'glucose_history_df' in globals():
        df = glucose_history_df.get(patient_id)
        if df is not None:
            return df.copy()
    # Synthesize a plausible trend based on seed for determinism
    rng = np.random.default_rng(seed=patient_id)
    t = pd.date_range(end=pd.Timestamp.today(), periods=60, freq='D')
    vals = np.clip(110 + np.cumsum(rng.normal(0, 5, size=len(t))), 70, 300)
    return pd.DataFrame({"timestamp": t, "glucose_mg_dL": vals})



In [ ]:
# === Model IO Utilities ===
from typing import Any, Dict

MODEL_PATH = os.path.join("models", "final_model.joblib")
PIPELINE_PATH = os.path.join("models", "pipeline_artifacts.joblib")


def save_model_and_pipeline(model: Any, artifacts: Dict[str, Any]) -> Dict[str, str]:
    dump(model, MODEL_PATH)
    dump(artifacts, PIPELINE_PATH)
    return {"model_path": MODEL_PATH, "pipeline_path": PIPELINE_PATH}


def load_model_and_pipeline() -> Dict[str, Any]:
    model = load(MODEL_PATH)
    artifacts = load(PIPELINE_PATH)
    return {"model": model, **artifacts}

# Try to snapshot artifacts from earlier cells if available
try:
    pipeline_artifacts = {
        "imputer": imputer,
        "selector": selector,
        "label_encoders": label_encoders if 'label_encoders' in globals() else {},
        "sanitized_feature_names": sanitized_feature_names,
        "selected_features": selected_features,
        "feature_name_mapping": feature_name_mapping,
    }
    if 'final_model' in globals():
        save_model_and_pipeline(final_model, pipeline_artifacts)
except Exception as _e:
    pass



In [72]:
# === Trend + Recommendation Agent (AutoGen-wrapped) ===
from dataclasses import dataclass

@dataclass
class TrendResult:
    image_path: str
    summary: str
    recommendation: str
    risk_factor: float


class TrendRecommendationAgent:
    def __init__(self, llm: LLMClient):
        self.llm = llm

    def _plot_trend(self, df: pd.DataFrame, patient_id: int) -> str | None:
        if df is None or df.empty or 'glucose_mg_dL' not in df.columns:
            return None
        fig, ax = plt.subplots(figsize=(8, 3))
        ax.plot(df['timestamp'], df['glucose_mg_dL'], color='tab:red')
        ax.set_title(f"Glucose Trend - Patient {patient_id}")
        ax.set_ylabel("mg/dL")
        ax.grid(True, alpha=0.3)
        path = f"static/trend_{patient_id}.png"
        fig.tight_layout()
        fig.savefig(path)
        plt.close(fig)
        return path

    def _model_risk_factor(self, X_selected: np.ndarray, model) -> float:
        try:
            prob = float(model.predict_proba(X_selected)[:, 1][0])
            return round(prob * 100.0, 1)
        except Exception:
            return 50.0

    def run(self, patient_id: int) -> Dict[str, Any]:
        # Build glucose trend if available
        df_trend = None
        try:
            df_trend = get_glucose_history(patient_id)
        except Exception:
            df_trend = None
        image_path = self._plot_trend(df_trend, patient_id)
        mean = float(df_trend['glucose_mg_dL'].mean()) if df_trend is not None and not df_trend.empty else None
        slope = float(np.polyfit(np.arange(len(df_trend)), df_trend['glucose_mg_dL'].values, 1)[0]) if df_trend is not None and len(df_trend) >= 2 else None

        # Load model and patient features for all-feature risk
        artifacts = load_model_and_pipeline()
        model = artifacts['model']
        feats = artifacts['selected_features']
        rev_map = {v: k for k, v in artifacts.get('feature_name_mapping', {}).items()}
        patient = get_patient_vector_from_id(patient_id)
        X_selected = patient['X_selected']
        risk = self._model_risk_factor(X_selected, model)

        # Snapshot of top features (from saved top-20 if available)
        top_names = artifacts.get('top_20_selected_names', feats[:10])
        # Reconstruct the imputed feature row aligned to feats
        try:
            # Recreate the imputed feature row
            row = df_train_enhanced[df_train_enhanced['patient_id'] == patient_id]
            X_full = row.drop(columns=['patient_id', 'readmitted', 'high_risk_90d'], errors='ignore').copy()
            def _sanitize_one(n: str) -> str:
                return artifacts.get('feature_name_mapping', {}).get(n, sanitize_feature_names([n])[0])
            X_full.columns = [_sanitize_one(c) for c in X_full.columns]
            X_full = X_full.reindex(columns=artifacts.get('sanitized_feature_names', feats), fill_value=0)
            for col, enc in artifacts.get('label_encoders', {}).items():
                if col in X_full.columns:
                    try:
                        X_full[col] = enc.transform(X_full[col].astype(str))
                    except Exception:
                        X_full[col] = enc.transform([enc.classes_[0]])[0]
            X_imp = pd.DataFrame(artifacts['imputer'].transform(X_full), columns=X_full.columns, index=X_full.index)
            row_imp = X_imp.iloc[0]
            top_snapshot = {rev_map.get(f, f): float(row_imp.get(f, np.nan)) for f in top_names}
        except Exception:
            top_snapshot = {}

        # LLM prompts now reference model risk and key feature snapshot
        trend_context = ""
        if mean is not None and slope is not None:
            trend_context = f"Mean glucose: {mean:.1f} mg/dL, slope: {slope:.3f}/day. Recent: " \
                            f"{df_trend['glucose_mg_dL'].tail(5).round(1).tolist()}"
        prompt_summary = (
            "Summarize this patient's overall risk context using model risk and key features.\n"
            f"Model risk score (0-100): {risk}.\n"
            f"Key feature values: {top_snapshot}.\n"
            + (f"Trend: {trend_context}" if trend_context else "")
        )
        summary = self.llm.generate(prompt_summary)

        prompt_reco = (
            "Provide 3-5 plain-language, actionable recommendations for diabetes care, "
            "grounded in the model risk and key features. Avoid diagnosis.\n"
            f"Risk (0-100): {risk}. Key features: {top_snapshot}."
            + (f" Trend: {trend_context}" if trend_context else "")
        )
        recommendation = self.llm.generate(prompt_reco)

        return {"trend_image_path": image_path, "trend_summary": summary, "recommendation": recommendation, "risk_factor": risk}

from autogen import AssistantAgent as _Assistant

class TrendAssistant(_Assistant):
    def __init__(self, name: str, llm_client: LLMClient):
        super().__init__(name)
        self.impl = TrendRecommendationAgent(llm_client)

    def handle(self, patient_id: int) -> Dict[str, Any]:
        return self.impl.run(patient_id)



In [ ]:
# # === CSV Loader and Weighted Target (replaces HF dataset) ===
# import pandas as pd
# import numpy as np

# CSV_PATH = "risk_prediction_10k_validated_GPU_with_additional_cols.csv"  # <-- set this

# # Map your CSV columns to expected names if needed
# COLUMN_MAP = {
#     # 'your_patient_id_col': 'patient_id',
#     # 'your_readmitted_col': 'readmitted',
#     # 'your_emergency_col': 'number_emergency',
#     # 'your_inpatient_col': 'number_inpatient',
#     # 'your_a1c_gt8_flag_col': 'A1Cresult:>8',
#     # 'your_glucose_gt300_flag_col': 'max_glu_serum:>300',
# }

# USE_CSV_ONLY = True

# print("📥 Loading CSV...")
# df_train = pd.read_csv(CSV_PATH)
# if COLUMN_MAP:
#     df_train = df_train.rename(columns=COLUMN_MAP)

# # Ensure required columns exist or create defaults
# if 'patient_id' not in df_train.columns:
#     df_train['patient_id'] = np.arange(1, len(df_train) + 1)

# for col, default in [
#     ('readmitted', 0),
#     ('number_emergency', 0),
#     ('number_inpatient', 0),
#     ('A1Cresult:>8', 0),
#     ('max_glu_serum:>300', 0),
# ]:
#     if col not in df_train.columns:
#         df_train[col] = default

# print(f"📊 CSV shape: {df_train.shape}")

# # Weighted risk target per your spec
# print("🔧 Computing weighted risk target from CSV...")
# weights = {
#     'readmitted': 0.40,
#     'number_emergency': 0.20,  # interpreted as high ER utilization flag (>0)
#     'number_inpatient': 0.15,  # flag for multiple admissions (>1)
#     'A1Cresult:>8': 0.15,
#     'max_glu_serum:>300': 0.10,
# }

# def create_risk_target_from_csv(df: pd.DataFrame) -> pd.Series:
#     score = np.zeros(len(df), dtype=float)
#     score += df['readmitted'].astype(float) * weights['readmitted']
#     score += (df['number_emergency'] > 0).astype(float) * weights['number_emergency']
#     score += (df['number_inpatient'] > 1).astype(float) * weights['number_inpatient']
#     score += df['A1Cresult:>8'].astype(float) * weights['A1Cresult:>8']
#     score += df['max_glu_serum:>300'].astype(float) * weights['max_glu_serum:>300']
#     return (score >= 0.5).astype(int)

# df_train['high_risk_90d'] = create_risk_target_from_csv(df_train)
# print(df_train['high_risk_90d'].value_counts(normalize=True))

# # Disable test set
# TEST_AVAILABLE = False

# # Rebuild enhanced features and proceed
# df_train_enhanced = create_advanced_clinical_features(df_train)
# print(f"📈 Training features after enhancement (CSV): {df_train_enhanced.shape[1]}")



📥 Loading CSV...
📊 CSV shape: (10000, 73)
🔧 Computing weighted risk target from CSV...
high_risk_90d
0    1.0
Name: proportion, dtype: float64
📈 Training features after enhancement (CSV): 80


In [ ]:
# === Enhanced SHAP Agent (saves top-20 plot) + Orchestrator v2 ===
from typing import Dict, Any, List, Tuple
import numpy as np
import os
import matplotlib.pyplot as plt

class EnhancedSHAPInsightAgent:
    def __init__(self, llm: LLMClient):
        self.llm = llm

    def _compute_shap(self, model, X_selected: np.ndarray) -> np.ndarray:
        try:
            import shap
            # Prefer TreeExplainer when available
            if hasattr(model, "get_booster") or hasattr(model, "estimators_"):
                explainer = shap.TreeExplainer(model)
                values = explainer.shap_values(X_selected)
                if isinstance(values, list):
                    values = values[1] if len(values) > 1 else values[0]
                return np.array(values).reshape(-1)
            # Fallback to KernelExplainer
            bg = np.tile(X_selected, (20, 1))
            explainer = shap.KernelExplainer(lambda X: model.predict_proba(X)[:, 1], bg)
            values = explainer.shap_values(X_selected, nsamples=100)
            return np.array(values).reshape(-1)
        except Exception:
            # Last-resort: zero vector
            return np.zeros(X_selected.shape[1], dtype=float)

    def run(self, patient_id: int) -> Dict[str, Any]:
        artifacts = load_model_and_pipeline()
        model = artifacts["model"]
        feats: List[str] = artifacts["selected_features"]
        rev_map = {v: k for k, v in artifacts.get("feature_name_mapping", {}).items()}

        patient = get_patient_vector_from_id(patient_id)
        X_selected = patient["X_selected"]

        shap_vals = self._compute_shap(model, X_selected)
        pairs = list(zip(feats, shap_vals))
        pairs_sorted = sorted(pairs, key=lambda x: abs(float(x[1])), reverse=True)
        top20 = pairs_sorted[:20]
        top20_display = [(rev_map.get(f, f), float(v)) for f, v in top20]

        # Save bar plot
        os.makedirs("static", exist_ok=True)
        shap_path = f"static/shap_{patient_id}.png"
        names = [n for n, _ in top20_display]
        vals = [v for _, v in top20_display]
        colors = ['#d62728' if v > 0 else '#1f77b4' for v in vals]
        fig, ax = plt.subplots(figsize=(8, 6))
        y = np.arange(len(names))
        ax.barh(y, vals, color=colors)
        ax.set_yticks(y)
        ax.set_yticklabels(names)
        ax.invert_yaxis()
        ax.set_title(f"Top-20 SHAP contributions - Patient {patient_id}")
        ax.set_xlabel("SHAP value (impact on risk)")
        fig.tight_layout()
        fig.savefig(shap_path, dpi=150)
        plt.close(fig)

        # LLM explanation using top-20
        table = "\n".join([f"- {n}: {v:+.4f}" for n, v in top20_display])
        prompt = (
            "Explain these top-20 factors affecting readmission risk in clear, non-technical language.\n"
            f"Top-20 (feature: shap):\n{table}\n"
            "Keep sentences short. Include both increases and decreases in risk."
        )
        explanation = self.llm.generate(prompt)

        return {"shap_insight": explanation, "top_contributions": top20_display, "shap_image_path": shap_path}

class OrchestratorAgentV2:
    def __init__(self, llm: LLMClient):
        self.shap_agent = EnhancedSHAPInsightAgent(llm)
        self.trend_agent = TrendRecommendationAgent(llm)

    def run(self, patient_id: int) -> Dict[str, Any]:
        shap_out = self.shap_agent.run(patient_id)
        trend_out = self.trend_agent.run(patient_id)
        return {
            "shap_insight": shap_out["shap_insight"],
            "shap_image_path": shap_out.get("shap_image_path"),
            "trend_image_path": trend_out["trend_image_path"],
            "trend_summary": trend_out["trend_summary"],
            "recommendation": trend_out["recommendation"],
            "risk_factor": trend_out["risk_factor"],
        }




In [77]:
orchestrator_v2 = OrchestratorAgentV2(llm_client)
report = orchestrator_v2.run(int(df_train_enhanced['patient_id'].iloc[0]))
report

{'shap_insight': "These factors influence the chance of a patient being readmitted to the hospital.  A positive number means higher risk; a negative number means lower risk.\n\n**Higher Risk Factors:**\n\n* **Many healthcare contacts:** More interactions with the healthcare system increase risk.\n* **Many emergency room visits:** Frequent ER visits signal higher risk.\n* **High utilizer:** Patients identified as high utilizers of healthcare have a higher readmission risk.\n* **Many previous hospital stays:** A history of many hospitalizations increases the chance of another.\n* **Many diagnoses:** More health problems increase the readmission risk.\n\n**Lower Risk Factors:**\n\n* **Orthopedic specialty:** Patients treated by orthopedic specialists have a lower risk.\n* **Fewer medications:** Taking fewer medications lowers the risk.\n* **Shorter hospital stays:** Shorter stays usually mean lower readmission risk.\n* **Emergency admission avoidance:** Not being admitted through the ER l

In [76]:
# === Generate 4 orchestrator V2 outputs for 4 different patients ===
from pprint import pprint

orchestrator_v2 = OrchestratorAgentV2(llm_client)
patient_ids = df_train_enhanced['patient_id'].drop_duplicates().astype(int).head(4).tolist()
outputs = []
for pid in patient_ids:
    try:
        out = orchestrator_v2.run(pid)
        outputs.append({"patient_id": pid, **out})
    except Exception as e:
        outputs.append({"patient_id": pid, "error": str(e)})

print("Generated 4 orchestrator reports:")
for o in outputs:
    pprint(o)



Generated 4 orchestrator reports:
{'patient_id': 1,
 'recommendation': 'Based on the provided data, here are some actionable '
                   'recommendations for improving diabetes care:\n'
                   '\n'
                   "1. **Medication Review:** You're currently taking 27 "
                   'medications.  This is a high number and may indicate a '
                   'need for a medication review with your doctor or '
                   'pharmacist.  They can help assess if all medications are '
                   'necessary and identify any potential interactions or '
                   'redundancies.  This could simplify your routine and '
                   'potentially reduce side effects.\n'
                   '\n'
                   '2. **Focus on Preventative Care:** Your history shows '
                   'multiple hospitalizations and emergency room visits.  '
                   'Proactive measures are crucial.  Discuss preventative '
                   'st